In [31]:
import os
import gc
import warnings
warnings.filterwarnings("ignore")

In [32]:
import evaluate
import torch
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from datasets import Dataset, DatasetDict
from transformers import logging, AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
logging.set_verbosity_error()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc.collect()
torch.manual_seed(42)

In [33]:
train_data = pd.read_csv('../dataset/full_train_data_summarization.csv')
validation_data = pd.read_csv('../dataset/full_validation_data_summarization.csv')
test_data = pd.read_csv('../dataset/full_test_data_summarization.csv')

In [5]:
validation_data = validation_data[:100]
test_data = test_data[:600]

In [5]:
model_name = 'google/mt5-base'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
prefix = "Hãy tóm tắt ngắn gọn nội dung sau bằng tiếng Việt: "
def preprocess_function(examples):
  inputs = [prefix + doc for doc in examples["context"]]
  model_inputs = tokenizer(inputs, max_length=4096, truncation=True)
  labels = tokenizer(text_target=examples["summarization"], max_length=1024, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [8]:
new_data = DatasetDict({
    "train": Dataset.from_dict(train_data),
    "validation": Dataset.from_dict(validation_data)
})

In [9]:
tokenized_new_data = new_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

In [11]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  bleu_scores_ngram_1 = []
  bleu_scores_ngram_2 = []
  bleu_scores_ngram_3 = []
  bleu_scores_ngram_4 = []
  bleu_scores_ngram_avg = []
  for reference_text, generated_text in zip(decoded_labels, decoded_preds):
    bleu_score_ngram_1 = sentence_bleu([reference_text], generated_text, weights=(1, 0, 0, 0))
    bleu_score_ngram_2 = sentence_bleu([reference_text], generated_text, weights=(0, 1, 0, 0))
    bleu_score_ngram_3 = sentence_bleu([reference_text], generated_text, weights=(0, 0, 1, 0))
    bleu_score_ngram_4 = sentence_bleu([reference_text], generated_text, weights=(0, 0, 0, 1))
    bleu_score_ngram_avg = sentence_bleu([reference_text], generated_text, weights=(0.25, 0.25, 0.25, 0.25))
    bleu_scores_ngram_1.append(bleu_score_ngram_1)
    bleu_scores_ngram_2.append(bleu_score_ngram_2)
    bleu_scores_ngram_3.append(bleu_score_ngram_3)
    bleu_scores_ngram_4.append(bleu_score_ngram_4)
    bleu_scores_ngram_avg.append(bleu_score_ngram_avg)

  return {
    'bleu@1': sum(bleu_scores_ngram_1) / len(bleu_scores_ngram_1),
    'bleu@2': sum(bleu_scores_ngram_2) / len(bleu_scores_ngram_2),
    'bleu@3': sum(bleu_scores_ngram_3) / len(bleu_scores_ngram_3),
    'bleu@4': sum(bleu_scores_ngram_4) / len(bleu_scores_ngram_4),
    'bleu@avg': sum(bleu_scores_ngram_avg) / len(bleu_scores_ngram_avg)
  }

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name.replace('/', '_').replace('-', '_')}_model_summarization",
    learning_rate=1e-5,
    auto_find_batch_size=True,
    # per_device_train_batch_size=4,
    # per_device_eval_batch_size=4,
    num_train_epochs=6,
    predict_with_generate=True,
    bf16=True,
    push_to_hub=False,
    save_total_limit=1,
    save_strategy='epoch',
    evaluation_strategy='epoch'
)

In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_new_data["train"],
    eval_dataset=tokenized_new_data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
torch.cuda.empty_cache()
gc.collect()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


0

In [15]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: phamduchuy159. Use `wandb login --relogin` to force relogin


{'loss': 5.9844, 'learning_rate': 9.722222222222223e-06, 'epoch': 0.17}
{'loss': 1.2591, 'learning_rate': 9.444444444444445e-06, 'epoch': 0.33}
{'loss': 1.0245, 'learning_rate': 9.166666666666666e-06, 'epoch': 0.5}
{'loss': 0.9405, 'learning_rate': 8.888888888888888e-06, 'epoch': 0.67}
{'loss': 0.8937, 'learning_rate': 8.611111111111112e-06, 'epoch': 0.83}
{'loss': 0.8691, 'learning_rate': 8.333333333333334e-06, 'epoch': 1.0}
{'eval_loss': 0.5431631207466125, 'eval_bleu@1': 0.001063575725282157, 'eval_bleu@2': 0.000799717288024215, 'eval_bleu@3': 0.0005921217204276331, 'eval_bleu@4': 0.00048222215393011164, 'eval_bleu@avg': 0.000683548001854629, 'eval_runtime': 47.9858, 'eval_samples_per_second': 6.252, 'eval_steps_per_second': 0.792, 'epoch': 1.0}
{'loss': 0.8209, 'learning_rate': 8.055555555555557e-06, 'epoch': 1.17}
{'loss': 0.842, 'learning_rate': 7.77777777777778e-06, 'epoch': 1.33}
{'loss': 0.8065, 'learning_rate': 7.500000000000001e-06, 'epoch': 1.5}
{'loss': 0.8043, 'learning_r

TrainOutput(global_step=18000, training_loss=0.9045706532796224, metrics={'train_runtime': 3962.344, 'train_samples_per_second': 9.086, 'train_steps_per_second': 4.543, 'train_loss': 0.9045706532796224, 'epoch': 6.0})

# Test Model Summarization

In [6]:
model_checkpoint = '../model_baseline/google_mt5_base_model_summarization/checkpoint-18000'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.to(device)
if torch.cuda.device_count() >= 2:
  model = torch.nn.DataParallel(model)

def generate_text(text):
  prefix = 'Hãy tóm tắt ngắn gọn nội dung sau bằng tiếng Việt: '
  encoding = tokenizer(prefix+text, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
  outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    early_stopping=False,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.8,
    repetition_penalty=1.2
  )
  for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    torch.cuda.empty_cache()
    return line

In [7]:
from tqdm import tqdm
tqdm.pandas()

In [10]:
test_data[f'generate_google_mt5'] = test_data['context'].progress_apply(lambda x: generate_text(x))

100%|██████████| 200/200 [10:00<00:00,  3.00s/it]


In [11]:
test_data

,context,summarization,generate_google_mt5
0,"Để khắc phục các nhược điểm nói trên, Viện Kho...",Viện Khoa học Fraunhofer Đức đang phát triển l...,"Viện Khoa học Fraunhofer, Đức đang phát triển ..."
1,Không nên dùng sản phẩm sát trùng mạnh như Bac...,Tránh dùng sản phẩm sát trùng mạnh. Tránh tran...,Tóm tắt ngắn gọn nội dung sau để vệ sinh khuyê...
2,"Kỳ 1: Đổi giờ học, giờ làm - đúng nhưng chưa đ...","Gần đây, Bộ Giao thông vận tải (GTVT) đã đưa r...",Bộ Giao thông vận tải (GTVT) đã đưa ra giải ph...
3,"Theo San Francisco Globe, chú hắc mã này có ng...",Frederik là một con ngựa Frieasian đến từ Hà L...,Ông Frederik là một người hâm mộ của vị vua tr...
4,Tham dự buổi làm việc có Thứ trưởng Bộ GTVT Ng...,Tham dự buổi làm việc có Thứ trưởng Bộ GTVT Ng...,Thứ trưởng Bộ GTVT Nguyễn Nhật và đại diện các...
...,...,...,...
195,"Trong giai đoạn 2015 2020, Đảng bộ VCCI đã đề ...",Đại hội Đảng bộ VCCI lần thứ VI nhiệm kỳ 2010-...,"Tại giai đoạn 2015-2020, Đảng bộ VCCI đã đề ra..."
196,Bản hit của nhạc sĩ Phạm Toàn Thắng đã có tổng...,Nhạc sĩ Phạm Toàn Thắng đã có bản hit có tổng ...,"Tổng hơn 13 triệu lượt view trong năm qua, tro..."
197,Các lực lượng chức năng đang tiến hành điều tr...,Lực lượng chức năng đang tiến hành điều tra vụ...,"Tại hầm đường bộ Phước Tượng, xe ô-tô tải Air ..."
198,"Kodaikanal, Tamil Nadu.\nMột điểm đến thân thi...","Tại Nam Ấn Độ, có nhiều điểm đến thân thiện gi...",Tìm về Kodaikanal ở Tamil Nadu. Nằm giữa những...


In [13]:
test_data.to_csv('test_google_mt5.csv', index=False)

In [6]:
test_google_mt5 = pd.read_csv('test_google_mt5.csv')

In [10]:
bleu_scores_ngram_1 = []
bleu_scores_ngram_2 = []
bleu_scores_ngram_3 = []
bleu_scores_ngram_4 = []
bleu_scores_ngram_avg = []
for i, row in test_google_mt5.iterrows():
    bleu_score_ngram_1 = sentence_bleu([row['context']], row['generate_google_mt5'], weights=(1, 0, 0, 0))
    bleu_score_ngram_2 = sentence_bleu([row['context']], row['generate_google_mt5'], weights=(0, 1, 0, 0))
    bleu_score_ngram_3 = sentence_bleu([row['context']], row['generate_google_mt5'], weights=(0, 0, 1, 0))
    bleu_score_ngram_4 = sentence_bleu([row['context']], row['generate_google_mt5'], weights=(0, 0, 0, 1))
    bleu_score_ngram_avg = sentence_bleu([row['context']], row['generate_google_mt5'], weights=(0.25, 0.25, 0.25, 0.25))
    bleu_scores_ngram_1.append(bleu_score_ngram_1)
    bleu_scores_ngram_2.append(bleu_score_ngram_2)
    bleu_scores_ngram_3.append(bleu_score_ngram_3)
    bleu_scores_ngram_4.append(bleu_score_ngram_4)
    bleu_scores_ngram_avg.append(bleu_score_ngram_avg)
bleu_scores = {
    'bleu@1': sum(bleu_scores_ngram_1) / len(bleu_scores_ngram_1),
    'bleu@2': sum(bleu_scores_ngram_2) / len(bleu_scores_ngram_2),
    'bleu@3': sum(bleu_scores_ngram_3) / len(bleu_scores_ngram_3),
    'bleu@4': sum(bleu_scores_ngram_4) / len(bleu_scores_ngram_4),
    'bleu@avg': sum(bleu_scores_ngram_avg) / len(bleu_scores_ngram_avg)
}

In [11]:
bleu_scores

{'bleu@1': 0.13707125511825846,
 'bleu@2': 0.12818132221785022,
 'bleu@3': 0.12191466015495132,
 'bleu@4': 0.11792501227617203,
 'bleu@avg': 0.12554119295645577}